# Bespoke MCC phase skew

## Use MCC peak-calls to link SNPs to CAD promoter regions
## Use bespoke WASP input files keyed off SNP haplotypes to assess gene-skew

In [7]:
import pandas as pd
import os
from glob import glob
import pysam

In [ ]:
# Files available on request from Matt Baxter
matt_directory = "<matt_directory>"
shared_directory = "<shared_directory>"
ed_directory="<ed_directory>"

cad_promoter_file_path = "/project/Wellcome_Discovery/mbaxter/00_MCC/00_Peak_calling/HUVEC_CAD_Promoters_1Mb.bed"

peak_calls_format_lookup = {
    "HUVEC_D1": matt_directory + "/00_MCC/00_Peak_calling/MCC5_HUVEC1_v2/{}_mcc_peaks.bed",
    "HUVEC_D2": matt_directory + "/00_MCC/00_Peak_calling/MCC3_HUVEC2_v2/{}_mcc_peaks.bed",
    "HUVEC_D3": matt_directory + "/00_MCC/00_Peak_calling/MCC6_HUVEC3_v2/{}_mcc_peaks.bed",   
}
backbone_vcf_file_path = shared_directory + "/01_in_house_data/02_HUVEC/00_WGS/00_PackBio/03_backbone/backbone.variants.vcf.gz"
phased_vcf_file_format = shared_directory + "/01_in_house_data/02_HUVEC/00_WGS/00_PackBio/01_phased_VCFs_no_RefCalls/{}_phased_variants.vcf.gz"
bam_counts_format=ed_directory + "/01_huvec_skew/04_bam_counts/{donor}.BAM_counts.vcf.gz"
point_bam_format= shared_directory + "/01_in_house_data/02_HUVEC/03_RNASeq/01_CATCH_ALL/HUVEC_ref/results/07_indices/{}-point_hg38.bam"


output_test_frame_filepath = ed_directory + "/01_huvec_skew/08_bespoke_wasp/linked.genes.tsv"

captured_huvec_enhancers_filepath = matt_directory + "/17_CAD_paper/tables/Captured_HUVEC_enhancers_for_ED.txt"
dbsnp_file_path = shared_directory + "/00_publicly_available/06_dbSNP/dbSNP.v156.hg38.vcf.gz"

wasp_input_format= ed_directory + "/01_huvec_skew/06_wasp_files/ATAC/{donor}.WASP.input.tsv"
wasp_output_filepattern = ed_directory + "/01_huvec_skew/06_wasp_files/{}/cht_results.txt"

gencode_filepath = shared_directory + "/00_publicly_available/11_gencode_annotation/gencode.v45.primary_assembly.annotation.gtf"

bespoke_wasp_input_format = ed_directory + "/01_huvec_skew/08_bespoke_wasp/{}.WASP.Input.2.tsv"

output_summary_filepath = ed_directory + "/01_huvec_skew/08_bespoke_wasp/results.genes.v4.tsv"

## Load in the CAD promoters and rsids of interest

In [4]:
# Load in precomputed promoter region to gene labels

cad_promoters = pd.read_csv(
    cad_promoter_file_path, sep="\t",
    header=None
)[list(range(4)) + [18]]

cad_promoters.columns = ["chrom", "start", "end", "regmen", "gene_name"]


In [5]:
rsids = []
for peak_call_format in peak_calls_format_lookup.values():
    rsids.extend([os.path.basename(f).split("_")[0] for f in glob(peak_call_format.format("*"))])

rsids = sorted(set(rsids))

print(len(rsids))
# print(rsids)

93


In [68]:
backbone_vcf = pysam.VariantFile(backbone_vcf_file_path)

donor_vcfs_for_phase = {
    donor: pysam.VariantFile(phased_vcf_file_format.format(donor)) for donor in ["HUVEC_D1", "HUVEC_D2", "HUVEC_D3"]
}

missing = 0
indels = 0
n_hets = 0
n_indel_hets = 0
n_no_linked_genes = 0


test_frame = []
for _, row in rsid_definitions.iterrows():
    variants = list(backbone_vcf.fetch(row.chrom, row.pos - 1, row.pos))
    
    peaks_lookup = {
        x: pd.read_csv(y.format(row.rsid), sep="\t") for x, y in peak_calls_format_lookup.items() if 
        os.path.exists(y.format(row.rsid))
    }
    if len(peaks_lookup) == 0:
        print("Missing for rsid ", row.rsid)
        continue

    # Use peak files to link SNPs to genes via CAD promoters
    linked_genes = set()
    for donor, peaks in peaks_lookup.items():
        for _, peak in peaks.iterrows():            
            cad_genes = cad_promoters[
                (cad_promoters.chrom == peak.Chromosome) &
                (cad_promoters.start <= peak.End + 100) &
                (cad_promoters.end >= peak.Start - 100)
            ]
            for gene_name in set(cad_genes.gene_name):
                linked_genes.add(gene_name)
    
    linked_genes = set(linked_genes).intersection(set(genes.gene_name))
    if len(linked_genes) == 0:
        n_no_linked_genes += 1
        continue


    # Build up the annotations to link variants to genes. Use WASP-style column labels
    if len(variants) == 1:
        assert variants[0].pos == row.pos
        if len(variants[0].ref) == len(variants[0].alts[0]):
            assert row.ref in {variants[0].ref, variants[0].alts[0]}
            assert row.alt in {variants[0].ref, variants[0].alts[0]}
        else:
            print(variants[0].ref, variants[0].alts[0])
            print(row.ref, row.alt)
            indels += 1
            if any((set(var.samples[don]["GT"]) == {0, 1} for don in donors)):
                n_indel_hets += 1
            print()
        var = variants[0]

        donors = [
            "HUVEC_D1", "HUVEC_D2", "HUVEC_D3",
        ]
        if any((set(var.samples[don]["GT"]) == {0, 1} for don in donors)):
            snp_label = {
               "CHROM": var.chrom,
                "TEST.SNP.POS": var.pos,
                "TEST.SNP.ID": var.id,
                "TEST.SNP.REF.ALLELE": var.ref,
                "TEST.SNP.ALT.ALLELE": var.alts[0],
            }
            for donor in donors:
                snp_label_donor = snp_label.copy()
                snp_label_donor["donor"] = donor

                if set(var.samples[donor]["GT"]) == {None}:
                    snp_label_donor["TEST.SNP.GENOTYPE"] = 0.0
                    snp_label_donor["TEST.SNP.HAPLOTYPE"] = "0|0"
                    snp_label_donor["phaseset"] = -1
                elif set(var.samples[donor]["GT"]) == {0, 1}:
                    snp_label_donor["TEST.SNP.GENOTYPE"] = 1.0
                    snp_label_donor["TEST.SNP.HAPLOTYPE"] = "|".join(
                        list(map(str, var.samples[donor]["GT"]))
                    )
                    
                    phase_vars = list(donor_vcfs_for_phase[donor].fetch(var.chrom, var.pos-1, var.pos))
                    assert len(phase_vars) == 1
                    phase_var = phase_vars[0]
                    assert phase_var.ref == var.ref and phase_var.alts == var.alts
                    
                    snp_label_donor["phaseset"] = phase_var.samples[donor].get("PS", -1)
                else:
                    assert set(var.samples[donor]["GT"]) == {1}
                    snp_label_donor["TEST.SNP.GENOTYPE"] = 2.0
                    snp_label_donor["TEST.SNP.HAPLOTYPE"] = "1|1"
                    snp_label_donor["phaseset"] = -1
                
                for gene in linked_genes:
                    snp_label_donor_gene = snp_label_donor.copy()
                    snp_label_donor_gene["gene"] = gene
                    test_frame.append(snp_label_donor_gene)
            
            n_hets += 1
            
            
        
    elif len(variants) == 0:
        missing += 1
    else:
        print("Have 2 here")
        pass
    
print(
    "missing, indels, n_hets, n_indel_hets, n_no_linked_genes=",
    missing, indels, n_hets, n_indel_hets, n_no_linked_genes
)
test_frame = pd.DataFrame(test_frame)
test_frame.to_csv(output_test_frame_filepath, sep="\t", index=False)
test_frame


GA G
A -

C CTT
- TT

A AAGT
- AGT

Missing for rsid  chr19_11085680_C_-
C CAG
- AG

missing, indels, n_hets, n_indel_hets, n_no_linked_genes= 3 4 37 4 32


,CHROM,TEST.SNP.POS,TEST.SNP.ID,TEST.SNP.REF.ALLELE,TEST.SNP.ALT.ALLELE,donor,TEST.SNP.GENOTYPE,TEST.SNP.HAPLOTYPE,phaseset,gene
0,chr1,2232129,rs263533,C,T,HUVEC_D1,1.0,0|1,587048,SKI
1,chr1,2232129,rs263533,C,T,HUVEC_D1,1.0,0|1,587048,FAAP20
2,chr1,2232129,rs263533,C,T,HUVEC_D2,1.0,1|0,588411,SKI
3,chr1,2232129,rs263533,C,T,HUVEC_D2,1.0,1|0,588411,FAAP20
4,chr1,2232129,rs263533,C,T,HUVEC_D3,1.0,1|0,589367,SKI
...,...,...,...,...,...,...,...,...,...,...
208,chr9,129168215,rs10819474,T,C,HUVEC_D1,0.0,0|0,-1,IER5L
209,chr9,129168215,rs10819474,T,C,HUVEC_D2,2.0,1|1,-1,NTMT1
210,chr9,129168215,rs10819474,T,C,HUVEC_D2,2.0,1|1,-1,IER5L
211,chr9,129168215,rs10819474,T,C,HUVEC_D3,1.0,1|0,101341845,NTMT1


In [11]:
# Use dbSNP to remap rsID labels

rsid_definitions = pd.read_csv(captured_huvec_enhancers_filepath, sep="\t", header=None)

rsid_definitions.columns = ["chrom", "pos", "ref", "alt", "rsid"]
db_snp = pysam.VariantFile(db_snp_filepath)
rsid_mapping = {}
for _, row in rsid_definitions.iterrows():
    db_def = [x for x in db_snp.fetch(row.chrom, row.pos-2, row.pos+2)]
    if row.rsid not in {x.id for x in db_def}:
        new_records = [x for x in db_def if x.pos == row.pos]
        if len(new_records) != 1:
            continue
        new_record = new_records[0]
        
        rsids.append(new_record.id)
        rsid_mapping[new_record.id] = row.rsid

rsids = sorted(set(rsids))

In [12]:
# Load in precomputed WASP output. This is the ATAC skew not central to CAD paper

common_cols = 'TEST.SNP.CHROM', 'TEST.SNP.POS', 'TEST.SNP.ID', 'TEST.SNP.REF.ALLELE','TEST.SNP.ALT.ALLELE', 'REGION.START', 'REGION.END'

data_type="ATAC"
wasp_output=pd.read_csv(wasp_output_filepattern.format(data_type), sep="\t")

other_cols = set(wasp_output.columns) - set(common_cols)
rename = {x: "{}.{}".format(x, data_type) for x in other_cols}

wasp_output = wasp_output.rename(columns=rename)
wasp_output["SKEW.{}".format(data_type)] = wasp_output["REF.AS.READ.COUNT.{}".format(data_type)] / wasp_output["TOTAL.AS.READ.COUNT.{}".format(data_type)]


# Use gene-code gene annotations to define POINT-seq regions

In [14]:
# Load in gene database

genes = pd.read_csv(gencode_filepath, skiprows=5, header=None, sep="\t")
genes = genes[genes[2] == "gene"]
genes = genes[genes[8].str.contains("protein")]
genes = genes[genes[0].str.startswith("chr")]

genes[9] = genes[8].str.split(";").apply(
    lambda x: [y for y in x if "gene_name" in y][0]
).str.split("gene_name").apply(lambda x: x[1]).str.strip().str.strip("\"")

genes = genes[[0, 3, 4, 6, 9]]

genes.columns = ["chrom", "start", "end", "strand", "gene_name"]

genes.head(n=2)

,chrom,start,end,strand,gene_name
58,chr1,65419,71585,+,OR4F5
314,chr1,450740,451678,-,OR4F29


In [15]:
# For all donors form grid of point values
# One VCf is only used for the phase count which isn't present for allele counts.

donors = ["HUVEC_D1", "HUVEC_D2", "HUVEC_D3"]

data = []

for donor in donors:
    print(donor)
    
    vcf = pysam.VariantFile(phased_vcf_file_format.format(donor))
    vcf_counts = pysam.VariantFile(bam_counts_format.format(donor=donor))

    for _, gene in genes.iterrows():
        strand = gene.strand == "+"

        vcf_variants = list(vcf.fetch(gene.chrom, gene.start, gene.end))
        vcf_variants_counts = list(vcf_counts.fetch(gene.chrom, gene.start, gene.end))

        phase = None
        point_allele1 = 0
        point_allele2 = 0
        
        rna_seq_allele1 = 0
        rna_seq_allele2 = 0
        
        for var, var_counts in zip(vcf_variants, vcf_variants_counts):
            if len(var.ref) > 1:
                continue
            if len(var.alts) > 1:
                continue
            if len(var.alts[0]) > 1:
                continue
            
            new_phase = var.samples[donor].get("PS")
            if new_phase is None:
                continue

            if phase is None:
                phase = new_phase
            elif phase != new_phase:
                break
            
            assert var.id == var_counts.id
            assert var.samples[donor]["GT"] == var_counts.samples[donor]["GT"]

            allele_code1 = "R" if var.samples[donor]["GT"] == (0, 1) else "A"
            allele_code2 = "A" if var.samples[donor]["GT"] == (0, 1) else "R"
            strand_code = "F" if strand else "R"
            
            point_allele1 += var_counts.info.get("P" + strand_code + allele_code1, 0)
            point_allele2 += var_counts.info.get("P" + strand_code + allele_code2, 0)
            
            rna_seq_allele1 += var_counts.info.get("R" + strand_code + allele_code1, 0)
            rna_seq_allele2 += var_counts.info.get("R" + strand_code + allele_code2, 0)
        

        data.append({
            "gene": gene.gene_name,
            "allele1": point_allele1,
            "allele2": point_allele2,
            "rna_seq_allele1": rna_seq_allele1,
            "rna_seq_allele2": rna_seq_allele2,
            "donor": donor,
            "phaseset": phase,
        })
    
point = pd.DataFrame(data)

point[point.gene == "MMP2"]

HUVEC_D1
HUVEC_D2
HUVEC_D3


,gene,allele1,allele2,rna_seq_allele1,rna_seq_allele2,donor,phaseset
14630,MMP2,137,140,766,771,HUVEC_D1,54252436.0
34679,MMP2,1183,1349,611,637,HUVEC_D2,46382627.0
54728,MMP2,546,535,331,342,HUVEC_D3,46382645.0


In [16]:
# Load in the pre-computed wasp input files for the 3 donors

donors = ["HUVEC_D{}".format(i) for i in range(1, 4)]

total = []
for donor in donors:
    wasp_input_files = pd.read_csv(wasp_input_format.format(donor=donor), sep=" ")
    wasp_input_files = wasp_input_files[wasp_input_files["TEST.SNP.ID"].isin(rsids)]
    wasp_input_files["donor"] = donor
    total.append(wasp_input_files)

wasp_input_files = pd.concat(total)
display(wasp_input_files)

,CHROM,TEST.SNP.POS,TEST.SNP.ID,TEST.SNP.REF.ALLELE,TEST.SNP.ALT.ALLELE,TEST.SNP.GENOTYPE,TEST.SNP.HAPLOTYPE,REGION.START,REGION.END,REGION.SNP.POS,REGION.SNP.HET.PROB,REGION.SNP.LINKAGE.PROB,REGION.SNP.REF.HAP.COUNT,REGION.SNP.ALT.HAP.COUNT,REGION.SNP.OTHER.HAP.COUNT,REGION.READ.COUNT,GENOMEWIDE.READ.COUNT,donor
235,chr1,2232129,rs263533,C,T,1.00,0|1,2231150,2233061,2231505;2232129;2232677,0.9815737349020663;0.9860587390115204;0.986721...,0.5;0.5;0.5,4;7;1,1;7;3,0;0;0,374,101614726,HUVEC_D1
2299,chr1,24705677,rs66530629,G,A,0.01,0|0,24704974,24706046,NaN,NaN,NaN,NaN,NaN,NaN,495,101614726,HUVEC_D1
2305,chr1,24724734,rs1316829,C,T,0.01,0|0,24724401,24725857,24724662,0.4891842560813785,0.5,6,10,0,467,101614726,HUVEC_D1
4033,chr1,56411837,rs11206803,C,T,0.01,0|0,56411187,56412269,NaN,NaN,NaN,NaN,NaN,NaN,407,101614726,HUVEC_D1
8518,chr1,175169466,rs10912901,G,A,0.01,0|0,175168803,175170501,NaN,NaN,NaN,NaN,NaN,NaN,1118,101614726,HUVEC_D1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119272,chr8,133546137,rs16904940,C,T,0.01,0|0,133545478,133546745,NaN,NaN,NaN,NaN,NaN,NaN,760,227790954,HUVEC_D3
121120,chr9,22103342,rs1537373,T,G,0.01,0|0,22103207,22103972,NaN,NaN,NaN,NaN,NaN,NaN,479,227790954,HUVEC_D3
123971,chr9,107755513,rs944172,C,T,0.01,0|0,107754953,107756029,NaN,NaN,NaN,NaN,NaN,NaN,649,227790954,HUVEC_D3
125157,chr9,127845239,rs11789185,T,G,2.00,1|1,127844614,127845884,NaN,NaN,NaN,NaN,NaN,NaN,914,227790954,HUVEC_D3


## Create the WASP input files

These each link activity rsid (in enhancer/promoter peak) to haplotype counts along gene

In [69]:
# Create the bespoke Huvec point libraries.
# These join together donors based upon shared snp genotypes in the reg activity.

from new_skew_wasp_input import get_haplotypes_counts, get_other_haplotype_counts
from io import StringIO

# Run for HUVEC_D1, HUVEC_D2, HUVEC_D3
donor = "HUVEC_D3"

# Need to load from both VCF files. First file has phasing information.
# Second file has precomputed counts information for all datasets including POINT-seq
vcf = pysam.VariantFile(phased_vcf_file_format.format(donor))
vcf_counts = pysam.VariantFile(bam_counts_format.format(donor=donor))

# Compute total read counts.
stats = pd.read_csv(
    StringIO(pysam.idxstats(
        point_bam_format.format(donor)
    )),
    sep="\t",
    header=None
)
genome_wide_read_count = stats[2].sum()
print(genome_wide_read_count)

output = []
atac_input = None
for _, row in test_frame[test_frame.donor == donor].iterrows():
    
    gene = genes[genes.gene_name == row.gene].iloc[0]
    
    read_count = pysam.AlignmentFile(point_bam_format.format(donor)).count(gene.chrom, gene.start, gene.end)

    # Load all variants from gene region
    variants = list(vcf.fetch(gene.chrom, gene.start, gene.end))
    variant_counts = list(vcf_counts.fetch(gene.chrom, gene.start, gene.end))
    assert len(variants) == len(variant_counts)
    
    # Restrict to only simply mono-allelic heterozygous SNPs
    variants = [v for v in variants if set(v.samples[donor]["GT"]) == {0, 1}]
    variant_counts = [v for v in variant_counts if set(v.samples[donor]["GT"]) == {0, 1}]
    
    # Restrict only to SNPs in phase with activity SNP. Using PacBio phasesets
    variants = [v for v in variants if v.samples[donor].get("PS") == row["phaseset"]]
    variant_counts = [v for v in variant_counts if v.samples[donor].get("PS") == row["phaseset"]]

    # Restrict to SNPs disregarding Indel counts wihtin Gene body
    variants = [v for v in variants if len(v.alts) == 1 and len(v.alts[0]) == len(v.ref)]
    variant_counts = [v for v in variant_counts if len(v.alts) == 1 and len(v.alts[0]) == len(v.ref)]
    assert len(variants) == len(variant_counts)
    
    # Get counts from forward stranded bam.
    experiment_key = "PF" if gene.strand == "+" else "PR"
    
    h1, h2 = get_haplotypes_counts(variant_counts, donor, experiment_key, True)
    other_counts = get_other_haplotype_counts(variant_counts, experiment_key)
    
    print(row["TEST.SNP.HAPLOTYPE"])
    if row["TEST.SNP.HAPLOTYPE"] == "0|1":
        ref_counts = h1
        alt_counts = h2
    else:
        ref_counts = h2
        alt_counts = h1
    
    # Write entry line for WASP input
    not_empty = len(variant_counts) > 0
    v_pos = [v.pos for v in variant_counts]
    adjusted_het_probs = [1.0 for v in variant_counts]
    hets_phase_probs = [1.0 for v in variant_counts]
    
    output.append({
        "CHROM": row["CHROM"],
        "TEST.SNP.POS": row["TEST.SNP.POS"],
        "TEST.SNP.ID": row["TEST.SNP.ID"],
        "TEST.SNP.REF.ALLELE": row["TEST.SNP.REF.ALLELE"],
        "TEST.SNP.ALT.ALLELE": row["TEST.SNP.ALT.ALLELE"],
        "TEST.SNP.GENOTYPE": row["TEST.SNP.GENOTYPE"],
        # Ideally this should include possibility not phased for clear understanding
        "TEST.SNP.HAPLOTYPE": row["TEST.SNP.HAPLOTYPE"],
        "REGION.START": gene.start,
        "REGION.END": gene.end,
        "REGION.SNP.POS": ";".join(list(map(str, v_pos))) if not_empty else "NA",

        "REGION.SNP.HET.PROB": ";".join(list(map(str, adjusted_het_probs))) if not_empty else "NA",
        "REGION.SNP.LINKAGE.PROB": ";".join(list(map(str, hets_phase_probs))) if not_empty else "NA",

        "REGION.SNP.REF.HAP.COUNT": ";".join(list(map(str, ref_counts))) if not_empty else "NA",
        "REGION.SNP.ALT.HAP.COUNT": ";".join(list(map(str, alt_counts))) if not_empty else "NA",
        "REGION.SNP.OTHER.HAP.COUNT": ";".join(list(map(str, other_counts))) if not_empty else "NA",

        "REGION.READ.COUNT": read_count,
        "GENOMEWIDE.READ.COUNT": genome_wide_read_count,
    })

output = pd.DataFrame(output)

output.to_csv(bespoke_wasp_input_format.format(donor), sep=" ", index=False)
output

428586390
1|0
1|0
1|1
0|1
0|1
0|1
1|0
1|0
1|0
0|0
0|1
0|0
1|0
1|0
1|0
1|0
1|0
1|0
1|0
1|0
0|1
0|1
0|1
0|1
0|0
1|0
1|0
1|0
1|1
1|1
1|1
0|0
1|0
1|0
1|1
1|1
0|0
0|0
0|0
1|0
0|0
1|0
1|0
1|0
1|0
1|0
1|0
1|0
1|1
1|1
1|0
1|1
1|1
0|0
1|0
1|0
0|0
0|1
0|1
0|1
0|0
0|0
1|0
0|0
0|0
0|0
0|0
0|0
0|0
1|0
1|0


,CHROM,TEST.SNP.POS,TEST.SNP.ID,TEST.SNP.REF.ALLELE,TEST.SNP.ALT.ALLELE,TEST.SNP.GENOTYPE,TEST.SNP.HAPLOTYPE,REGION.START,REGION.END,REGION.SNP.POS,REGION.SNP.HET.PROB,REGION.SNP.LINKAGE.PROB,REGION.SNP.REF.HAP.COUNT,REGION.SNP.ALT.HAP.COUNT,REGION.SNP.OTHER.HAP.COUNT,REGION.READ.COUNT,GENOMEWIDE.READ.COUNT
0,chr1,2232129,rs263533,C,T,1.0,1|0,2227388,2310213,2230051;2231034;2231505;2232129;2232677;223326...,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,111;194;217;118;148;6;150;198;78;121;160;163;2...,130;194;145;135;167;3;120;170;61;112;126;143;1...,1;0;4;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;1;0;1;0;...,300944,428586390
1,chr1,2232129,rs263533,C,T,1.0,1|0,2184461,2212720,2185716;2186900;2186920;2194537;2195995;220161...,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,4;2;1;11;4;0;0;0;3;3;3;0;3;1;3;2;4;0;2,9;6;5;11;1;0;0;0;2;2;1;0;4;1;1;1;2;0;4,0;0;0;3;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,11727,428586390
2,chr1,56411837,rs11206803,C,T,2.0,1|1,56494761,56645301,NA,NA,NA,NA,NA,NA,33410,428586390
3,chr1,205141789,rs3902193,C,A,1.0,0|1,205227946,205285632,205230014;205230312;205231592;205232197;205234...,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,4;1;0;0;2;2;1;1;2;0;1;1;1;3;1;6;6;0;1;4;0;7;2;...,2;1;2;1;2;2;1;0;2;2;2;1;1;1;1;8;2;1;1;2;2;5;2;...,0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;...,3039,428586390
4,chr1,205141789,rs3902193,C,A,1.0,0|1,205142505,205211702,205142633;205145537;205145971;205149059;205149...,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,5;2;3;5;3;2;2;11;2;5;3;11;18;1;1;0;3;0;4;8;9;1...,2;3;7;7;4;2;1;13;2;5;3;4;16;4;1;0;6;1;4;6;9;11...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;0;...,17576,428586390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,chr6,108902728,rs11961585,G,A,0.0,0|0,108848416,108974476,NA,NA,NA,NA,NA,NA,6670,428586390
67,chr8,133540375,rs59413766,C,A,0.0,0|0,133454848,133571926,NA,NA,NA,NA,NA,NA,169811,428586390
68,chr8,133546137,rs16904940,C,T,0.0,0|0,133454848,133571926,NA,NA,NA,NA,NA,NA,169811,428586390
69,chr9,129168215,rs10819474,T,C,1.0,1|0,129608884,129636135,129608952;129611051;129611734;129611887;129612...,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1.0;1....,1;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0;...,0;0;1;1;0;0;0;1;1;0;0;1;1;0;0;0;0;0;0;0;0;0;0;...,0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;...,7009,428586390


## Analysis of results. Reformat for Matt's spreadsheet

This second part of the notebook is to be run after wasp is run

In [33]:
genes["REGION"] = genes["chrom"] + ":" + genes["start"].astype(str) + "-" + genes["end"].astype(str)

bespoke_wasp = None
bespoke_wasp = pd.read_csv("/project/Wellcome_Discovery/esanders/01_huvec_skew/08_bespoke_wasp/cht_results.2.txt", sep="\t")

bespoke_wasp["REGION"] = bespoke_wasp["TEST.SNP.CHROM"] + ":" + bespoke_wasp["REGION.START"].astype(str) + "-" + bespoke_wasp["REGION.END"].astype(str)

print(bespoke_wasp.shape)

bespoke_wasp = bespoke_wasp.merge(genes, on="REGION")

bespoke_wasp.to_csv(output_summary_filepath, sep="\t", index=False)
bespoke_wasp

(71, 19)


,TEST.SNP.CHROM,TEST.SNP.POS,TEST.SNP.ID,TEST.SNP.REF.ALLELE,TEST.SNP.ALT.ALLELE,REGION.START,REGION.END,LOGLIKE.NULL,LOGLIKE.ALT,CHISQ,...,TOTAL.AS.READ.COUNT,REF.AS.READ.COUNT,ALT.AS.READ.COUNT,TOTAL.READ.COUNT,REGION,chrom,start,end,strand,gene_name
0,chr1,2232129,rs263533,C,T,2227388,2310213,-18459.27,-18407.60,103.337,...,26958,14619,12339,485295,chr1:2227388-2310213,chr1,2227388,2310213,+,SKI
1,chr1,2232129,rs263533,C,T,2184461,2212720,-250.36,-248.84,3.037,...,361,162,199,18281,chr1:2184461-2212720,chr1,2184461,2212720,-,FAAP20
2,chr1,56411837,rs11206803,C,T,56494761,56645301,-101.83,-101.79,0.096,...,151,71,80,130707,chr1:56494761-56645301,chr1,56494761,56645301,-,PLPP3
3,chr1,205141789,rs3902193,C,A,205227946,205285632,-131.92,-130.94,1.966,...,192,86,106,3418,chr1:205227946-205285632,chr1,205227946,205285632,+,TMCC2
4,chr1,205141789,rs3902193,C,A,205142505,205211702,-1067.08,-1066.39,1.366,...,1537,744,793,32457,chr1:205142505-205211702,chr1,205142505,205211702,-,DSTYK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,chr6,108902728,rs11961585,G,A,108848416,108974476,-4.16,-3.83,0.654,...,6,2,4,14077,chr6:108848416-108974476,chr6,108848416,108974476,+,ARMC2
67,chr8,133540375,rs59413766,C,A,133454848,133571926,-4142.14,-4139.65,4.978,...,5974,2870,3104,336809,chr8:133454848-133571926,chr8,133454848,133571926,-,ST3GAL1
68,chr8,133546137,rs16904940,C,T,133454848,133571926,-4142.14,-4139.65,4.978,...,5974,2870,3104,336809,chr8:133454848-133571926,chr8,133454848,133571926,-,ST3GAL1
69,chr9,129168215,rs10819474,T,C,129608884,129636135,-154.34,-154.02,0.647,...,224,121,103,11413,chr9:129608884-129636135,chr9,129608884,129636135,+,NTMT1
